# Data Cleaning Notebook

We choose to make a slope graph of the Male to Female Ratio for selected majors within 4 colleges inside UIUC. We chose this type of graph because it shows the change in gender ratio over a large period of time rather than every year which makes for a less aesthetic visualization (in my opinion)

## Data Cleaning Process
1. Grouped data by college then year. 
2. Filtered data for our selected majors within each college. Our criteria for majors was size and how well known it is (ie everyone knows mathematics is a major but not everyone knows the Agricultural Communications major. For our own ease, we selected the start year as 2004 because many majors were moved into different colleges in the years prior and 2004 is the year that Computer Science was moved from College of Business to College of Engineering. 
3. Fill all N/A values with 0
4. We calculated the M/F Ratio and wrote the DataFrame for each college to its respective csv. We opted for individual files for each college for ease of loading into d3 as pandas is much MUCH easier to use than javascript.

Note: Sorry for the messy code.

In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 4000
colleges = ["ACES", "Business", "LAS", "Engineering"]
years = [1980, 2018]
top_majors_list = [["Human Development & Family St", "Agr & Consumer Economics", "Animal Sciences", "Crop Sciences", "Nutritional Sciences", 
                   "Food Science & Human Nutrition", "Horticulture", "Natural Resrcs & Environ Sci"], 
               ["Business Administration (MBA)", "Business Administration", "Finance", "Curric Unassigned", "Nondegree"],
               ["Psychology", "Biology", "Molecular and Cellular Biology", "Political Science", "Mathematics", "History",
                   "Chemistry", "Chemical Engineering", "Economics"],
               ["Computer Science", "Electrical Engineering", "Computer Engineering", "Mechanical Engineering", "Civil Engineering",
                   "Aerospace Engineering", "Materials Science & Engr", "Bioengineering", "Industrial Engineering", "Physics"]]

data_list = ['Fall', 'Term Code', 'College', 'Department', 'Degree', 'Major Code',
       'Major_Name', 'Concentration Code', 'Concentration Name', 'Total',
       'Male', 'Female', 'Unknown', 'Caucasian', 'Asian American',
       'African American', 'Hispanic', 'Native American',
       'Hawaiian/Pacific Isl', 'Multiracial', 'International', 'Unknown.1',
       'All African American', 'All Native American', 'All Hawaiian/ Pac Isl',
       'All Asian', 'Illinois', 'Non-Illinois', 'Academic Program Code',
       'Legacy Advising Code', 'Legacy Department Code', 'Legacy Major Name']

important_cols = ['Fall', 'College', 'Major_Name', 'Total', 'Male', 'Female', 'Caucasian', 
                  'Asian American','African American', 'Hispanic', 'Native American']

majors = {colleges[i] : top_majors_list[i] for i in range(len(colleges))}

In [3]:
df2 = pd.read_csv('uiuc-students-by-curriculum.csv', thousands=',', low_memory=False)
df2.rename(columns={'Major Name': 'Major_Name'}, inplace=True)
df2 = df2[important_cols]
df2 = df2.loc[df2.Fall >= years[0]] #make sure only years we want
df2 = df2.loc[df2.Fall <= years[1]]
print(df2.shape[0])
print(df2.dtypes)

17453
Fall                  int64
College              object
Major_Name           object
Total                object
Male                float64
Female               object
Caucasian           float64
Asian American       object
African American     object
Hispanic            float64
Native American      object
dtype: object


In [4]:
toFloat = ['Total', 'Female', 'Male', 'Caucasian', 'Asian American', 'African American', 'Hispanic', 'Native American']
abnorm = ['Art and Design', 'Graphic Design']
#fix the data types
df2.fillna('0')
for i in toFloat:
    if df2[i].dtype == float: continue
    df2 = df2.loc[df2[i] != abnorm[0]]
    df2 = df2.loc[df2[i] != abnorm[1]]
    df2[i] = df2[i].str.replace(',','').astype(float).fillna(0.0)
print(df2.dtypes)
df2[:5]

Fall                  int64
College              object
Major_Name           object
Total               float64
Male                float64
Female              float64
Caucasian           float64
Asian American      float64
African American    float64
Hispanic            float64
Native American     float64
dtype: object


,Fall,College,Major_Name,Total,Male,Female,Caucasian,Asian American,African American,Hispanic,Native American
0,1981,Business,Accountancy,1433.0,816.0,617.0,1303.0,37.0,68.0,15.0,3.0
1,1981,Business,Accountancy,172.0,109.0,63.0,118.0,9.0,1.0,NaN,1.0
2,1982,Business,Accountancy,1408.0,773.0,635.0,1279.0,30.0,70.0,21.0,2.0
3,1982,Business,Accountancy,146.0,89.0,57.0,106.0,6.0,1.0,NaN,1.0
4,1983,Business,Accountancy,1392.0,703.0,689.0,1251.0,37.0,73.0,20.0,1.0


In [5]:
#combine duplicates that match year college and major
df2 = df2.groupby(['Fall', 'College', 'Major_Name'])[toFloat].sum().reset_index()
print(df2[:5])
df2['Major_Name'] = df2['Major_Name'].apply(str.title)
df2['College'] = df2['College'].apply(str.title)
df2['M/F'] = round(df2['Male'] / df2['Female'],2)
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna(axis=0) #to remove things witth broken ratio
df2 = df2.loc[df2.Total >= 20] #more than 20 people
df2[:5]

   Fall College                      Major_Name  Total  Female   Male  \
0  1980    ACES  Agr & Environmental Cmc & Educ    5.0     1.0    4.0   
1  1980    ACES     Agricultural & Applied Econ  315.0    38.0  277.0   
2  1980    ACES     Agricultural Communications   79.0    48.0   31.0   
3  1980    ACES          Agricultural Education   26.0     5.0   21.0   
4  1980    ACES        Agricultural Engineering   83.0     8.0   75.0   

   Caucasian  Asian American  African American  Hispanic  Native American  
0        4.0             0.0               0.0       0.0              0.0  
1      282.0             3.0               4.0       1.0              1.0  
2       78.0             0.0               0.0       0.0              1.0  
3       25.0             0.0               1.0       0.0              0.0  
4       79.0             1.0               1.0       1.0              0.0  


,Fall,College,Major_Name,Total,Female,Male,Caucasian,Asian American,African American,Hispanic,Native American,M/F
1,1980,Aces,Agricultural & Applied Econ,315.0,38.0,277.0,282.0,3.0,4.0,1.0,1.0,7.29
2,1980,Aces,Agricultural Communications,79.0,48.0,31.0,78.0,0.0,0.0,0.0,1.0,0.65
3,1980,Aces,Agricultural Education,26.0,5.0,21.0,25.0,0.0,1.0,0.0,0.0,4.20
4,1980,Aces,Agricultural Engineering,83.0,8.0,75.0,79.0,1.0,1.0,1.0,0.0,9.38
5,1980,Aces,Agricultural Sciences,81.0,24.0,57.0,77.0,2.0,2.0,0.0,0.0,2.38


In [6]:
majorToCol_map = {}
fix_list = []
doesnt_exist = []
df2 = df2.sort_values(['Fall'], ascending = [False])
for i in df2.iterrows():
    if i[1]['Fall'] == (2018): 
        majorToCol_map[i[1]['Major_Name']] = i[1]['College']

In [7]:
#if major isnt still around get rid of
for i in df2.iterrows():
    temp = i[1]['Major_Name']
    if temp not in majorToCol_map:
        doesnt_exist.append(int(i[0]))
    elif majorToCol_map[temp] != i[1]['College']: 
        fix_list.append((int(i[0]), i[1]['Major_Name']))
  
for i in doesnt_exist:
    df2.drop(i, inplace = True)

In [8]:
#Make sure everything is under the right college
for i in fix_list:
    df2.at[i[0],'College'] = majorToCol_map[i[1]]
print(df2.shape[0])

3774


In [9]:
cur_majors = set(majorToCol_map.keys())
for i in range(years[1], years[0]-1):
    seen_majors = set(df2[df2.Fall == i]['Major_Names'])
    to_add = cur_majors - seen_majors
    for j in to_add:
        pd.dataFrat(i, colToMajor_Map[j], j, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [10]:
df2.to_csv('cleaned_project2_data.csv')

PermissionError: [Errno 13] Permission denied: 'cleaned_project2_data.csv'

In [11]:
for c in ["Aces", "Business", "Las", "Engineering", "Education", "Applied Health Sciences", "Fine And Applied Arts", "Media"]:
    new_drame = df2.loc[df2['College'] == c]
    new_drame.to_csv('./dataframe' + c + '.csv')
df2.to_csv('./cleaned.csv', index=False)